In [5]:
import numpy as np
import pandas as pd
import pickle
import os
import tensorflow as tf
import alibi
import matplotlib.pyplot as plt
from alibi.explainers import IntegratedGradients
# import matplotlib.pyplot as plt
import shap

all_features = []
feature_name = ['p90','p95','p80','p5','p10','p20','range_90_to_80','range_20_to_10','range_90_to_10',
                'range_80_to_20','median','iqr','skew']
name = 'current_window_stress'
all_features += [name+'_'+a for a in feature_name]
# print(len(set(all_features)),len(all_features))
name = 'current_window_stress_diff'
all_features += [name+'_'+a for a in feature_name]
name = 'daily_stress'
all_features += [name+'_'+a for a in feature_name]
all_features += ['percentage_of_stress', 'maximum_duration_of_current_stress_episode','average_duration_of_current_stress_episode',
               'maximum_density_of_current_stress_episode', 'average_density_of_current_stress_episode', 'average_deviation_to_daily_mean_current',
               'no_stress_till_now', 'time_since_last_stress', 'duration_of_last_stress_episode', 'average_duration_of_before_stress_episode',
               'density_of_last_stress_episode', 'average_density_of_before_stress_episode', 'deviation_to_daily_mean_of_last_stress_episode',
               'percentage_of_stress_before']
all_features+= ['percentage_of_active_', 'maximum_duration_of_current_activity_episode', 'average_duration_of_current_window_episode',
                 'no_activity_till_now', 'time_since_last_activity', 'duration_of_last_activity_episode', 'average_duration_of_before_activity_episode',
                 'percentage_of_active_before']
all_features += ['time_since_last_visit_smoking_spot','duration_of_stay_in_smoking_spot','fraction_of_minutes_spent_in_smoking_spots_out_of_observed']
all_features += ['Gender','Age']
all_features += ['is_smoking','spread','distance_to_nearest_spot',
                'time_spent_in_transition','time_spent_in_smoking_spot']
all_features += ['Time of day = {}'.format(a) for a in np.arange(24)]
feature_names = all_features

In [6]:
def set_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
    # Restrict TensorFlow to only use the first GPU
        try:
            tf.config.experimental.set_visible_devices(gpus[gpu_id], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        except RuntimeError as e:
            # Visible devices must be set before GPUs have been initialized
            print(e)
gpu_id = 0
set_gpu()
path = './explannation_models/'
df_all = []
path+=os.listdir(path)[0]+'/'
import uuid
for iter in os.listdir(path):
    model_files = os.listdir(path+iter)
    data_dict,X_test,y_test = pickle.load(open(path+iter+'/data.p','rb'))
    all_models = []
    for weight_path in list(data_dict.keys()):
        with open(path+iter+'/model.json','r') as json_file:
            model = tf.keras.models.model_from_json(json_file.read())
        model.load_weights(weight_path)
        # for i, layer in enumerate(model.layers):
        #     layer._name = 'layer_' + str(uuid.uuid4())
        model._name=str(uuid.uuid4())
        ig  = IntegratedGradients(model,
                          method="gausslegendre",
                          n_steps=50,
                          internal_batch_size=100)
        explanation = ig.explain(X_test,
                                baselines=None,
                                target=None)

        attributions = explanation.attributions
        # shap.summary_plot([attributions[y_test==1].mean(axis=1)],X_test[y_test==1].mean(axis=1),feature_names=feature_names)
        # shap.summary_plot([attributions.mean(axis=1)],X_test.mean(axis=1),feature_names=feature_names)
        feature_importance = np.abs(attributions.sum(axis=1)).mean(axis=0)
        df = pd.DataFrame({'importance':feature_importance,
                           'name':feature_names,
                           'weight':[weight_path]*len(feature_importance),
                           'iter':[iter]*len(feature_importance)})
        print(df.sort_values('importance').reset_index(drop=True)[-5:]['name'].values)
        df_all.append(df)
        pickle.dump(pd.concat(df_all),open('./data/ig_dataframe1.p','wb'))
        # all_models.append(model)
    # input_ = all_models[0].input
    # model_outputs = [model(input_) for model in all_models]
    # output = tf.keras.layers.Average()(model_outputs)
    # ensemble_model = tf.keras.Model(inputs=input_, outputs=output)


4 Physical GPUs, 1 Logical GPU


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.
It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


['average_duration_of_before_activity_episode'
 'average_density_of_before_stress_episode'
 'time_since_last_visit_smoking_spot' 'daily_stress_iqr'
 'percentage_of_stress_before']


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


['daily_stress_range_80_to_20' 'time_since_last_activity'
 'time_since_last_stress' 'average_duration_of_before_activity_episode'
 'percentage_of_stress_before']


...

['average_density_of_before_stress_episode'
 'current_window_stress_diff_range_90_to_80' 'distance_to_nearest_spot'
 'daily_stress_p90' 'duration_of_stay_in_smoking_spot']


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


['distance_to_nearest_spot' 'daily_stress_range_90_to_80'
 'percentage_of_stress_before' 'duration_of_stay_in_smoking_spot'
 'time_since_last_visit_smoking_spot']


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


['time_since_last_activity' 'distance_to_nearest_spot'
 'deviation_to_daily_mean_of_last_stress_episode'
 'percentage_of_stress_before' 'Time of day = 15']
['daily_stress_p80' 'daily_stress_range_90_to_80'
 'fraction_of_minutes_spent_in_smoking_spots_out_of_observed'
 'duration_of_stay_in_smoking_spot' 'daily_stress_skew']


In [ ]:
# 
# plt.hist(model.predict(X_test)[y_test==0])
feature_importance.shape

In [ ]:

ig  = IntegratedGradients(model,
                          method="gausslegendre",
                          n_steps=50,
                          internal_batch_size=100)
explanation = ig.explain(X_test,
                         baselines=None,
                         target=None)

attributions = explanation.attributions

In [ ]:
# import matplotlib.pyplot as plt
import shap

all_features = []
feature_name = ['p90','p95','p80','p5','p10','p20','range_90_to_80','range_20_to_10','range_90_to_10',
                'range_80_to_20','median','iqr','skew']
name = 'current_window_stress'
all_features += [name+'_'+a for a in feature_name]
# print(len(set(all_features)),len(all_features))
name = 'current_window_stress_diff'
all_features += [name+'_'+a for a in feature_name]
name = 'daily_stress'
all_features += [name+'_'+a for a in feature_name]
all_features += ['percentage_of_stress', 'maximum_duration_of_current_stress_episode','average_duration_of_current_stress_episode',
               'maximum_density_of_current_stress_episode', 'average_density_of_current_stress_episode', 'average_deviation_to_daily_mean_current',
               'no_stress_till_now', 'time_since_last_stress', 'duration_of_last_stress_episode', 'average_duration_of_before_stress_episode',
               'density_of_last_stress_episode', 'average_density_of_before_stress_episode', 'deviation_to_daily_mean_of_last_stress_episode',
               'percentage_of_stress_before']
all_features+= ['percentage_of_active_', 'maximum_duration_of_current_activity_episode', 'average_duration_of_current_window_episode',
                 'no_activity_till_now', 'time_since_last_activity', 'duration_of_last_activity_episode', 'average_duration_of_before_activity_episode',
                 'percentage_of_active_before']
all_features += ['time_since_last_visit_smoking_spot','duration_of_stay_in_smoking_spot','fraction_of_minutes_spent_in_smoking_spots_out_of_observed']
all_features += ['Gender','Age']
all_features += ['is_smoking','spread','distance_to_nearest_spot',
                'time_spent_in_transition','time_spent_in_smoking_spot']
all_features += ['Time of day = {}'.format(a) for a in np.arange(24)]
feature_names = all_features

In [ ]:
shap.summary_plot([attributions.mean(axis=1)],X_test.mean(axis=1),feature_names=feature_names)

In [ ]:
# plt.hist(y_pred[y_test==1])
# plt.hist(y_pred[y_test==0])

In [ ]:
attributions.shape